In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder

In [26]:
class VideosDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, label_encoder, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
        self.label_encoder = label_encoder
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error opening image {img_name}: {e}")
            return None, None

        annotation_name = img_name.replace('.jpg', '.csv')
        annotation_path = os.path.join(self.annotation_dir, annotation_name)

        try:
            annotation = pd.read_csv(annotation_path)
            if annotation.empty:
                raise ValueError(f"Annotation file {annotation_name} is empty.")
        except Exception as e:
            annotation = None

        if annotation is None:
            label_str = "Unknown" # Label for unknown class
        else:
            label_str = annotation.iloc[0, 4]  # Assuming label is in the second column
        
        label = self.label_encoder.transform([label_str])[0]  # Encode label
        if self.transform:
            image = self.transform(image)

        return image, label



In [27]:
image_dir = 'train/images2'
annotation_dir = 'train/labels2'

annotations = []
for filename in os.listdir(annotation_dir):
    if filename.endswith('.csv'):
        filepath = os.path.join(annotation_dir, filename)
        df = pd.read_csv(filepath)
        if df.iloc[:, 4].empty:
            annotations.extend(['Unknown'])
        else:
            annotations.extend(df.iloc[:, 4].values)

label_encoder = LabelEncoder()
label_encoder.fit(annotations)

LabelEncoder()

In [36]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = VideosDataset(image_dir='train/images2',
                              annotation_dir='train/labels2',
                              label_encoder=label_encoder,
                              transform=transform)

val_dataset = VideosDataset(image_dir='val/images',
                            annotation_dir='val/labels',
                            label_encoder=label_encoder,
                            transform=transform)

print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Train dataset size: 1027
Validation dataset size: 87


In [37]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

num_classes = len(label_encoder.classes_)
model = SimpleCNN(num_classes)

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
num_epochs = 25

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch+1}/{num_epochs}')
    model.train()
    running_loss = 0.0
    i = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(f'Batch {i+1}/{len(train_loader)} done. Loss: {loss.item()}')
        i += 1

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%')

Starting Epoch 1/25
Batch 1/17 done. Loss: 2.2901339530944824
Batch 2/17 done. Loss: 2.620124101638794
Batch 3/17 done. Loss: 1.30582857131958
Batch 4/17 done. Loss: 1.5204282999038696
Batch 5/17 done. Loss: 1.3721671104431152
Batch 6/17 done. Loss: 1.3804177045822144
Batch 7/17 done. Loss: 1.3035565614700317
Batch 8/17 done. Loss: 1.3273992538452148
Batch 9/17 done. Loss: 1.7527577877044678
Batch 10/17 done. Loss: 1.1418412923812866
Batch 11/17 done. Loss: 1.2780424356460571
Batch 12/17 done. Loss: 1.3498390913009644
Batch 13/17 done. Loss: 1.2545571327209473
Batch 14/17 done. Loss: 1.2262122631072998
Batch 15/17 done. Loss: 1.1440236568450928
Batch 16/17 done. Loss: 0.9280404448509216
Batch 17/17 done. Loss: 2.645277976989746
Epoch [1/25], Loss: 1.5200380963437699
Validation Loss: 1.290475070476532, Accuracy: 72.41379310344827%
Starting Epoch 2/25
Batch 1/17 done. Loss: 1.2507829666137695
Batch 2/17 done. Loss: 1.34011709690094
Batch 3/17 done. Loss: 1.1053386926651
Batch 4/17 done. 

KeyboardInterrupt: 

In [40]:
 # Évaluation finale sur le jeu de validation
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Final Validation Loss: {val_loss/len(val_loader)}, Final Accuracy: {100 * correct / total}%')

# Sauvegarde du modèle
torch.save(model.state_dict(), 'simple_cnn.pth')

Final Validation Loss: 1.4701437950134277, Final Accuracy: 72.41379310344827%
